In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import shape
import shapefile
import requests
from urllib import request
from zipfile import ZipFile
from io import BytesIO
from datetime import date
import osr

In [ ]:
year_list = pd.date_range('2002-01-01',date.today(), 
              freq='YS').strftime("%Y").tolist()
site_url = 'https://rmgsc.cr.usgs.gov/outgoing/GeoMAC/historic_fire_data/'
perimeter_url = 'https://rmgsc.cr.usgs.gov/outgoing/GeoMAC/historic_fire_data/US_HIST_FIRE_PERIMTRS_DD83.zip'

In [ ]:
geo_list = list()
for year in year_list[:-1]:
    zip_file_url = site_url+year+"_sit_rep_pts_dd83.zip"
    zipfile = ZipFile(BytesIO(request.urlopen(zip_file_url).read()))
    filenames = [y for y in sorted(zipfile.namelist()) for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)] 
    dbf, prj, shp, shx = [BytesIO(zipfile.read(filename)) for filename in filenames]
    r = shapefile.Reader(shp=shp, shx=shx, dbf=dbf)
    attributes, geometry = [], []
    field_names = [field[0] for field in r.fields[1:]]  
    for row in r.shapeRecords():  
        geometry.append(shape(row.shape.__geo_interface__))  
        attributes.append(dict(zip(field_names, row.record)))
        
    try:
        proj4_string = osr.SpatialReference(str(prj.read())).ExportToProj4()
        year_data = gpd.GeoDataFrame(data = attributes, geometry = geometry, crs = proj4_string)
    except:
        year_data = gpd.GeoDataFrame(data = attributes, geometry = geometry)
    
    geo_list.append(year_data)

In [ ]:
geo_list[0]

In [ ]:
# Perimeters
zipfile = ZipFile(BytesIO(request.urlopen(perimeter_url).read()))
filenames = [y for y in sorted(zipfile.namelist()) for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)] 
dbf, prj, shp, shx = [BytesIO(zipfile.read(filename)) for filename in filenames]
r = shapefile.Reader(shp=shp, shx=shx, dbf=dbf)
attributes, geometry = [], []
field_names = [field[0] for field in r.fields[1:]]  
for row in r.shapeRecords():  
    geometry.append(shape(row.shape.__geo_interface__))  
    attributes.append(dict(zip(field_names, row.record)))
        
try:
    proj4_string = osr.SpatialReference(str(prj.read())).ExportToProj4()
    perimeters = gpd.GeoDataFrame(data = attributes, geometry = geometry, crs = proj4_string)
except:
    perimeters = gpd.GeoDataFrame(data = attributes, geometry = geometry)

In [ ]:
print(perimeters.shape)
print(perimeters.columns)
perimeters.tail()

In [ ]:
# Do we have fires for every year?
perimeters.year_.value_counts()

In [ ]:
# Looks like a fire perimeter plot in the Geometry column:
perimeters.iloc[23471,9]